In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('dev_data_norm_round_spikesplit.csv')
abilities_df = pd.read_csv('abilities.csv')

In [13]:
abilities_df['agent_name'] = abilities_df['agent_name'].str.lower()

# Mapping dictionary
mapping = {'1': 1, '2': 2, '3': 3, 'ultimate': 4}

# Replace values using the mapping
abilities_df['ability'] = abilities_df['ability'].replace(mapping)
abilities_df

,agent_name,ability,crowd_control_general,crowd_control_mobility,crowd_control_vision,damage_for_team,damage_for_self,shield,information,healing
0,astra,1,0,0,0,1,0,0,0,0
1,astra,2,0,1,1,0,0,0,0,0
2,astra,3,0,0,1,0,0,0,0,0
3,astra,4,0,0,0,0,0,1,0,0
4,breach,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
79,viper,4,0,0,0,1,0,0,0,0
80,yoru,1,0,0,1,0,0,0,0,0
81,yoru,2,0,0,1,0,0,0,0,0
82,yoru,3,0,0,0,0,0,0,1,0


In [14]:
df_copy = df.copy()
df_copy

,user_id,map,player,ally4_character,self_character,ally1_character,ally2_character,ally3_character,opponent5_character,opponent6_character,...,self_pre_spike_total_loadout_value_loss,self_post_spike_total_loadout_value_loss,self_pre_spike_total_ability_usage_1,self_post_spike_total_ability_usage_1,self_pre_spike_total_ability_usage_2,self_post_spike_total_ability_usage_2,self_pre_spike_total_ability_usage_3,self_post_spike_total_ability_usage_3,self_pre_spike_total_ability_usage_4,self_post_spike_total_ability_usage_4
0,1006,Pearl,1,phoenix,jett,gekko,cypher,astra,harbor,astra,...,1834.0,0.0,0,0,1,0,1,0,0,0
1,1006,Pearl,1,phoenix,jett,gekko,cypher,astra,harbor,astra,...,7700.0,0.0,1,0,0,0,1,0,0,0
2,1006,Pearl,1,phoenix,jett,gekko,cypher,astra,harbor,astra,...,8350.0,3600.0,1,0,1,0,1,0,0,0
3,1006,Pearl,1,phoenix,jett,gekko,cypher,astra,harbor,astra,...,2136.0,0.0,1,0,0,0,1,0,0,0
4,1006,Pearl,1,phoenix,jett,gekko,cypher,astra,harbor,astra,...,800.0,0.0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8040,1018,Lotus,2,killjoy,sage,viper,breach,yoru,fade,harbor,...,12052.0,1600.0,1,1,3,1,1,2,0,0
8041,1018,Lotus,2,killjoy,sage,viper,breach,yoru,fade,harbor,...,8772.0,0.0,1,0,2,0,2,0,0,0
8042,1018,Lotus,2,killjoy,sage,viper,breach,yoru,fade,harbor,...,9160.0,0.0,3,0,2,0,1,0,0,0
8043,1018,Lotus,2,killjoy,sage,viper,breach,yoru,fade,harbor,...,4600.0,3500.0,0,2,2,2,1,0,0,0


In [15]:
# Create a set of the abilities characteristics columns
ability_cols = set(['crowd_control_general', 'crowd_control_mobility', 'crowd_control_vision', 
                    'damage_for_team', 'damage_for_self', 'shield', 'information', 'healing'])

# Initialize new columns to 0
for ability_col in ability_cols:
    df_copy[f"pre_spike_{ability_col}_used"] = 0
    df_copy[f"post_spike_{ability_col}_used"] = 0

# Iterate over the rows of the new_df dataframe
for idx, row in df_copy.iterrows():
    agent = row['self_character']
    
    for col in df_copy.columns:
        if 'ability_usage' in col:
            # Determine if the ability was used pre or post spike
            prepost = 'pre' if 'pre' in col else 'post'
            ab_num = int(col.split('ability_usage_')[1])

            # Only proceed if the ability was used
            if row[col] > 0:
                # Get the specific row from abilities_df
                agent_abilities_row = abilities_df[(abilities_df['agent_name'] == agent) & (abilities_df['ability'] == ab_num)]

                # For each ability characteristic column in the abilities dataframe
                for ability_col in ability_cols:
                    # If the ability has the characteristic, add a column in the main dataframe
                    if agent_abilities_row[ability_col].values[0] == 1:
                        new_col_name = f"{prepost}_spike_{ability_col}_used"
                        # Assign the number of times the ability was used to the new column
                        df_copy.at[idx, new_col_name] = row[col]

In [16]:
df_copy.iloc[0][250:]

self_pre_spike_total_ability_usage_2      1
self_post_spike_total_ability_usage_2     0
self_pre_spike_total_ability_usage_3      1
self_post_spike_total_ability_usage_3     0
self_pre_spike_total_ability_usage_4      0
self_post_spike_total_ability_usage_4     0
pre_spike_shield_used                     0
post_spike_shield_used                    0
pre_spike_damage_for_self_used            1
post_spike_damage_for_self_used           0
pre_spike_healing_used                    0
post_spike_healing_used                   0
pre_spike_crowd_control_vision_used       0
post_spike_crowd_control_vision_used      0
pre_spike_crowd_control_mobility_used     0
post_spike_crowd_control_mobility_used    0
pre_spike_damage_for_team_used            0
post_spike_damage_for_team_used           0
pre_spike_information_used                0
post_spike_information_used               0
pre_spike_crowd_control_general_used      0
post_spike_crowd_control_general_used     0
Name: 0, dtype: object

In [17]:
filtered_df = abilities_df[abilities_df['agent_name'] == 'phoenix']
filtered_df

,agent_name,ability,crowd_control_general,crowd_control_mobility,crowd_control_vision,damage_for_team,damage_for_self,shield,information,healing
52,phoenix,1,0,0,0,1,0,0,0,0
53,phoenix,2,0,0,1,0,0,0,0,0
54,phoenix,3,0,0,0,0,1,0,0,0
55,phoenix,4,0,0,0,0,0,0,0,1


In [18]:
df_copy.to_csv("dev_data_abilities.csv", index=False)